In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import csv
import os

COLOR_FONDO = "#e6ffe6"  # Color de fondo verde menta

# ---------- Funciones de datos ----------
def cargar_libros():
    libros = []
    if os.path.exists("libros.csv"):
        with open("libros.csv", newline='', encoding='utf-8') as archivo:
            lector = csv.reader(archivo)
            libros = list(lector)
    return libros

def guardar_libro(titulo, autor, genero, anio):
    with open("libros.csv", 'a', newline='', encoding='utf-8') as archivo:
        escritor = csv.writer(archivo)
        escritor.writerow([titulo, autor, genero, anio])
    messagebox.showinfo("Éxito", "Libro guardado correctamente.")

# ---------- Ventana de gestión de libros ----------
def ventana_libros():
    ventana = tk.Toplevel()
    ventana.title("Gestión de Libros")
    ventana.geometry("700x400")
    ventana.configure(bg=COLOR_FONDO)

    # Campos de entrada
    ttk.Label(ventana, text="Título:", background=COLOR_FONDO).grid(row=0, column=0)
    titulo = ttk.Entry(ventana)
    titulo.grid(row=0, column=1)

    ttk.Label(ventana, text="Autor:", background=COLOR_FONDO).grid(row=1, column=0)
    autor = ttk.Entry(ventana)
    autor.grid(row=1, column=1)

    ttk.Label(ventana, text="Género:", background=COLOR_FONDO).grid(row=2, column=0)
    genero = ttk.Entry(ventana)
    genero.grid(row=2, column=1)

    ttk.Label(ventana, text="Año:", background=COLOR_FONDO).grid(row=3, column=0)
    anio = ttk.Entry(ventana)
    anio.grid(row=3, column=1)

    # Tabla de libros
    tabla = ttk.Treeview(ventana, columns=("Título", "Autor", "Género", "Año"), show="headings")
    tabla.heading("Título", text="Título")
    tabla.heading("Autor", text="Autor")
    tabla.heading("Género", text="Género")
    tabla.heading("Año", text="Año")
    tabla.grid(row=6, column=0, columnspan=3, padx=10, pady=10)

    def actualizar_tabla():
        tabla.delete(*tabla.get_children())
        for libro in cargar_libros():
            tabla.insert("", tk.END, values=libro)

    def limpiar_campos():
        titulo.delete(0, tk.END)
        autor.delete(0, tk.END)
        genero.delete(0, tk.END)
        anio.delete(0, tk.END)

    def guardar():
        guardar_libro(titulo.get(), autor.get(), genero.get(), anio.get())
        limpiar_campos()
        actualizar_tabla()

    def seleccionar_libro(event):
        seleccionado = tabla.focus()
        if seleccionado:
            valores = tabla.item(seleccionado, 'values')
            titulo.delete(0, tk.END)
            autor.delete(0, tk.END)
            genero.delete(0, tk.END)
            anio.delete(0, tk.END)
            titulo.insert(0, valores[0])
            autor.insert(0, valores[1])
            genero.insert(0, valores[2])
            anio.insert(0, valores[3])

    def editar():
        seleccionado = tabla.focus()
        if not seleccionado:
            messagebox.showwarning("Atención", "Selecciona un libro para editar.")
            return
        valores = tabla.item(seleccionado, 'values')
        libros = cargar_libros()
        nuevo = [titulo.get(), autor.get(), genero.get(), anio.get()]
        libros = [nuevo if libro == list(valores) else libro for libro in libros]
        with open("libros.csv", 'w', newline='', encoding='utf-8') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerows(libros)
        messagebox.showinfo("Éxito", "Libro editado correctamente.")
        limpiar_campos()
        actualizar_tabla()

    def eliminar():
        seleccionado = tabla.focus()
        if not seleccionado:
            messagebox.showwarning("Atención", "Selecciona un libro para eliminar.")
            return
        valores = tabla.item(seleccionado, 'values')
        libros = cargar_libros()
        libros = [libro for libro in libros if libro != list(valores)]
        with open("libros.csv", 'w', newline='', encoding='utf-8') as archivo:
            escritor = csv.writer(archivo)
            escritor.writerows(libros)
        messagebox.showinfo("Éxito", "Libro eliminado correctamente.")
        limpiar_campos()
        actualizar_tabla()

    # Botones
    ttk.Button(ventana, text="Guardar libro", command=guardar).grid(row=4, column=0, pady=10)
    ttk.Button(ventana, text="Editar libro", command=editar).grid(row=4, column=1)
    ttk.Button(ventana, text="Eliminar libro", command=eliminar).grid(row=4, column=2)
    ttk.Button(ventana, text="Refrescar tabla", command=actualizar_tabla).grid(row=5, column=0, columnspan=3)

    tabla.bind("<ButtonRelease-1>", seleccionar_libro)
    actualizar_tabla()

# ---------- Menú principal ----------
def menu_principal():
    raiz = tk.Tk()
    raiz.title("Gestor de Biblioteca")
    raiz.geometry("600x400")
    raiz.configure(bg=COLOR_FONDO)

    marco = tk.Frame(raiz, bg=COLOR_FONDO)
    marco.place(relx=0.5, rely=0.5, anchor="center")

    ttk.Label(marco, text="Bienvenida al Gestor de Biblioteca", font=("Arial", 14), background=COLOR_FONDO).pack(pady=10)
    ttk.Button(marco, text="Gestionar Libros", command=ventana_libros).pack(pady=5)
    ttk.Button(marco, text="Salir", command=raiz.quit).pack(pady=5)

    raiz.mainloop()

menu_principal()